In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/dengue-temperatura-e-chuvas-em-campinassp/dengue-dataset.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df[df['chuva'].isnull()]

In [ ]:
df[['ano','mes','dia']] = df['data'].str.split('-', expand = True)

In [ ]:
df

In [ ]:
df.groupby('mes').mean()['chuva']

Preenchendo os valores null de chuva com a média do respectivo mês


In [ ]:
df.loc[[18, 126]] = df.loc[[18, 126]].fillna(47.38)

In [ ]:
df.loc[[19, 79, 115, 151, 175]] = df.loc[[19, 79, 115, 151, 175]].fillna(32.20)

In [ ]:
df.loc[53] = df.loc[53].fillna(45.70)

In [ ]:
df.loc[[18, 19, 53, 79, 115, 126, 151, 175]]

In [ ]:
df[['ano', 'mes', 'dia']] = df[['ano', 'mes', 'dia']].apply(pd.to_numeric)

In [ ]:
df.info()

In [ ]:
df.describe()

removendo outliers para o modelo, considerando valores acima de ((3 x std) + mean)

In [ ]:
df = df.drop(df[df['casos-confirmados'] > 5580].index)

In [ ]:
df.groupby('mes').mean()['casos-confirmados']

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.barplot(x='mes',y='casos-confirmados',data=df,estimator=np.mean,ci=False)

In [ ]:
sns.barplot(x='mes',y='chuva',data=df,estimator=np.mean,ci=False)

In [ ]:
sns.barplot(x='mes',y='temperatura-media',data=df,estimator=np.mean,ci=False)

In [ ]:
sns.barplot(x='ano',y='casos-confirmados',data=df,estimator=np.mean,ci=False)
plt.xticks(rotation = 90);

In [ ]:
sns.barplot(x='ano',y='chuva',data=df,estimator=np.mean,ci=False)
plt.xticks(rotation = 90);

In [ ]:
sns.barplot(x='ano',y='temperatura-media',data=df,estimator=np.mean,ci=False)
plt.xticks(rotation = 90);

Podemos ver pelos gráficos acima que aparentemente não há um claro padrão entre temperaturas e chuvas relacionado com o número de casos confirmados.

Criando uma variável categorica baseada na distribuição do número de casos confirmados

In [ ]:
def risco(casos):
    if casos<100:
        return 'baixo risco'
    elif casos<200 :
        return 'medio risco'
    else:
        return 'alto risco'

In [ ]:
df['risco']=df['casos-confirmados'].apply(risco)

In [ ]:
df.head()

In [ ]:
df['risco'].value_counts()

In [ ]:
X = df.drop(['data', 'risco','ano'], axis=1)

In [ ]:
y = df['risco']

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [ ]:
X_train = scaler.fit_transform(X_train)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
param_grid = {'C':[0.01,0.1,1,2,3,4,5], 'kernel':['linear', 'rbf']}

In [ ]:
svm = SVC(class_weight='balanced')

In [ ]:
svm_model=GridSearchCV(svm, param_grid)

In [ ]:
svm_model.fit(X_train, y_train)

In [ ]:
svm_model.best_params_

In [ ]:
y_pred = svm_model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,plot_confusion_matrix

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
plot_confusion_matrix(svm_model, X_test, y_test)

In [ ]:
print(classification_report(y_test, y_pred))

Mesmo não havendo uma clara relação entre chuva e temperatura, relacionadas com o número de casos, a junção dessas variáveis com os meses do ano possibilitou a criação de um modelo com 99% de accuracy e f1-score variando de 1 a 0.86 a depender da categoria.